<a href="https://colab.research.google.com/github/fatemeh-ict/NLp/blob/main/decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import math
from torch import nn
import torch.nn.functional as F

In [23]:

def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    print(f"scaled.size() : {scaled.size()}")
    if mask is not None:
        print(f"-- ADDING MASK of shape {mask.size()} --")
        # Broadcasting add. So just the last N dimensions need to match
        scaled += mask
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

In [25]:

class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        print('x after first linear layer:' ,x.size())
        x = self.relu(x)
        print('after relu layer:',x.size())
        x = self.dropout(x)
        print("x after dropout layer:" ,x.size())
        x = self.linear2(x)
        print("x after 2nd linear layer:", x.size())
        return x



In [26]:
class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta =  nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):

        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        print('dims', dims)
        mean = inputs.mean(dim=dims, keepdim=True)
        print("Mean:", mean.size())
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        print("Std:", std.size())
        y = (inputs - mean) / std
        print("y: ",y.size())
        out = self.gamma * y  + self.beta
        print("out:",out.size())
        return out

In [27]:
class MultiHeadAttention(nn.Module):

    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model , 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        batch_size, sequence_length, d_model = x.size()
        print("x.size:", x.size())
        qkv = self.qkv_layer(x)
        print("qkv.size:", qkv.size())
        qkv = qkv.reshape(batch_size, sequence_length, self.num_heads, 3 * self.head_dim)
        print("qkv after reshape size:", qkv.size())
        qkv = qkv.permute(0, 2, 1, 3)
        print("qkv after permutation:", qkv.size())
        q, k, v = qkv.chunk(3, dim=-1)
        print("q:" ,q.size(), "k:",k.size(), "v:",v.size())
        values, attention = scaled_dot_product(q, k, v, mask)
        print("values:", values.size(), "attention:",attention.size())
        values = values.reshape(batch_size, sequence_length, self.num_heads * self.head_dim)
        print("values after reshaping:", values.size())
        out = self.linear_layer(values)
        print("out after passing through linear layer:", out.size())
        return out

In [37]:
class MultiHeadCrossAttention(nn.Module):

    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = nn.Linear(d_model , 2 * d_model)
        self.q_layer = nn.Linear(d_model , d_model)
        self.linear_layer = nn.Linear(d_model, d_model)

    def forward(self, x, y, mask=None):
        batch_size, sequence_length, d_model = x.size()
        print("x.size:" ,x.size())
        kv = self.kv_layer(x)
        print("kv.size:" ,kv.size())
        q = self.q_layer(y)
        print("q.size:", q.size())
        kv = kv.reshape(batch_size, sequence_length, self.num_heads, 2 * self.head_dim)
        q = q.reshape(batch_size, sequence_length, self.num_heads, self.head_dim)
        kv = kv.permute(0, 2, 1, 3)
        q = q.permute(0, 2, 1, 3)
        k, v = kv.chunk(2, dim=-1)
        values, attention = scaled_dot_product(q, k, v, mask)
        print("values:", values.size(), "attention:",attention.size())
        values = values.reshape(batch_size, sequence_length, d_model)
        out = self.linear_layer(values)
        print("out after passing through linear layer:", out.size())
        return out

In [38]:
class DecoderLayer(nn.Module):

    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.encoder_decoder_attention = MultiHeadCrossAttention(d_model=d_model, num_heads=num_heads)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm3 = LayerNormalization(parameters_shape=[d_model])
        self.dropout3 = nn.Dropout(p=drop_prob)

    def forward(self, x, y, decoder_mask):
        _y = y
        print("mask self attention")
        y = self.self_attention(y, mask=decoder_mask)
        print("dropout1")
        y = self.dropout1(y)
        print("add layer norm1")
        y = self.norm1(y + _y)

        _y = y
        print("cross attention")
        y = self.encoder_decoder_attention(x, y, mask=None)
        print("dropout2")
        y = self.dropout2(y)
        print("add layer norm2")
        y = self.norm2(y + _y)

        _y = y
        print("feed forward1")
        y = self.ffn(y)
        print("dropout3")
        y = self.dropout3(y)
        print("add layer norm3")
        y = self.norm3(y + _y)
        return y

In [39]:
class SequentialDecoder(nn.Sequential):
    def forward(self, *inputs):
        x, y, mask = inputs
        for module in self._modules.values():
            y = module(x, y, mask)
        return y


In [40]:
class Decoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers=1):
        super().__init__()
        self.layers = SequentialDecoder(*[DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                          for _ in range(num_layers)])

    def forward(self, x, y, mask):

        y = self.layers(x, y, mask)
        return y

In [41]:
d_model = 512
num_heads = 8
drop_prob = 0.1
batch_size = 30
max_sequence_length = 200
ffn_hidden = 2048
num_layers = 5

In [42]:
x = torch.randn( (batch_size, max_sequence_length, d_model) )
y = torch.randn( (batch_size, max_sequence_length, d_model) )

mask = torch.full([max_sequence_length, max_sequence_length] , float('-inf'))
mask = torch.triu(mask, diagonal=1)
decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)
out = decoder(x, y, mask)

mask self attention
x.size: torch.Size([30, 200, 512])
qkv.size: torch.Size([30, 200, 1536])
qkv after reshape size: torch.Size([30, 200, 8, 192])
qkv after permutation: torch.Size([30, 8, 200, 192])
q: torch.Size([30, 8, 200, 64]) k: torch.Size([30, 8, 200, 64]) v: torch.Size([30, 8, 200, 64])
scaled.size() : torch.Size([30, 8, 200, 200])
-- ADDING MASK of shape torch.Size([200, 200]) --
values: torch.Size([30, 8, 200, 64]) attention: torch.Size([30, 8, 200, 200])
values after reshaping: torch.Size([30, 200, 512])
out after passing through linear layer: torch.Size([30, 200, 512])
dropout1
add layer norm1
dims [-1]
Mean: torch.Size([30, 200, 1])
Std: torch.Size([30, 200, 1])
y:  torch.Size([30, 200, 512])
out: torch.Size([30, 200, 512])
cross attention
x.size: torch.Size([30, 200, 512])
kv.size: torch.Size([30, 200, 1024])
q.size: torch.Size([30, 200, 512])
scaled.size() : torch.Size([30, 8, 200, 200])
values: torch.Size([30, 8, 200, 64]) attention: torch.Size([30, 8, 200, 200])
out af